In [6]:
#! pip install -qq mem0ai==0.0.9

In [7]:
import os
from openai import OpenAI
from mem0 import Memory
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")


client = OpenAI(api_key=OPENAI_API_KEY)

# https://pypi.org/project/qdrant-client/

# from qdrant_client import QdrantClient

# qdrant_client = QdrantClient(
#     url=QDRANT_URL,
#     api_key=QDRANT_API_KEY,
# )

system_prompt = """As a movie lover, you watch a lot of films. You are also multi-lingual.
You will receive: (a) a question about a movie, and (b) the language the user want the answer.
You can also translate the information into a language other than English if needed.
The goal is to provide information that will answer the user's question in the preferred language. 

If the user's query or ask:
- is ambigious, take the more common interpretation, or provide multiple interpretations and analysis.
- cannot be answered by the question, politely explain why.
- is not relevant to the movie topic, politely decline and explain that this tool assist in movies trivia only.

When responding to the user:
- avoid technical language, and always be succinct.
- avoid markdown header formatting
- do not reference any follow-up (e.g. "you may ask me further questions") as the conversation ends once you have completed your reply.

YOU MUST NEVER INCLUDE ANY MARKDOWN URLS  IN YOUR REPLY.


"""

In [8]:
# Initialize Mem0 
# https://docs.mem0.ai/overview
# memory = Memory()
# Define a user ID
#user_id = "john_doe"

# response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": "ps i love you"}
#             ],
#             temperature=0.2, 
#             max_tokens = 100
#         )

# resp = response.choices[0].message.content
# # Add user data to memory
# memory.add(resp, user_id=user_id)
# print("User data added to memory.")
# memory.get_all(user_id=user_id)


In [9]:
config = {
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "url": QDRANT_URL,
            "api_key": QDRANT_API_KEY
        }
    },
}
memory = Memory.from_config(config)

In [10]:
user_id = 'alice'
conversation_history = []

def get_ai_response(prompt):
    relevant_memories = memory.search(query=prompt, user_id=user_id)
    context = "Relevant past information:\n"
    for mem in relevant_memories:
        context += f"- {mem['text']}\n"
    
    full_prompt = f"{context}\nHuman: {prompt}\nAI:"
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        stream=True,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_prompt}
        ],
        temperature=0.2,
        max_tokens=100
    )
    
    answer = ""
    for chunk in response:
        chunk_content = chunk.choices[0].delta.content
        if chunk_content:
            answer += chunk_content
            print(chunk_content, end='', flush=True)
    
    print()  # New line after the complete answer
    return answer


while True:
    prompt = input("You: ")
    if prompt.lower() in ['exit', 'quit', 'bye']:
        break
    
    # Add user prompt to memory
    memory.add(f"User asked: {prompt}", user_id=user_id)
    
    # Get and print AI response
    answer = get_ai_response(prompt)
    
    # Add AI response to memory
    memory.add(f"AI answered: {answer}", user_id=user_id)
    
    # Add to conversation history
    conversation_history.append({"question": prompt, "answer": answer})

# Print conversation history
print("\nConversation History:")
for i, entry in enumerate(conversation_history, 1):
    print(f"\nExchange {i}:")
    print(f"Q: {entry['question']}")
    print(f"A: {entry['answer']}")



Educating Rita is a play written by Willy Russell that premiered in 1980. It tells the story of Rita, a working-class woman who enrolls in an Open University course to improve her life and education. She is paired with Frank, a disillusioned professor, and their relationship evolves as they challenge each other's perspectives on life, education, and social class. The play explores themes of self-discovery, personal growth, and the impact of education on one's identity. 

If you have a
I'm here to assist with movie-related questions only. If you have any inquiries about films, feel free to ask!

Conversation History:

Exchange 1:
Q: educating rita
A: Educating Rita is a play written by Willy Russell that premiered in 1980. It tells the story of Rita, a working-class woman who enrolls in an Open University course to improve her life and education. She is paired with Frank, a disillusioned professor, and their relationship evolves as they challenge each other's perspectives on life, educa

In [12]:
memory.get_all(user_id=user_id)

[{'id': '14a3a338-d572-4fde-ae60-349cb274e50c',
  'text': 'Interested in movies',
  'metadata': {'created_at': 1721671918,
   'data': 'Interested in movies',
   'user_id': 'alice'}},
 {'id': '8669cd4c-be46-47e9-9790-69d4c4276265',
  'text': "Interested in the play 'Educating Rita', which is written by Willy Russell. The play premiered in 1980 and tells the story of Rita, a working-class woman who enrolls in an Open University course to improve her life and education. She is paired with Frank, a disillusioned professor, and their relationship evolves as they challenge each other's perspectives on life, education, and social class. The play explores themes of self-discovery, personal growth, and the impact of education on one's identity.",
  'metadata': {'data': "Interested in the play 'Educating Rita', which is written by Willy Russell. The play premiered in 1980 and tells the story of Rita, a working-class woman who enrolls in an Open University course to improve her life and education

In [ ]:
# Using the "with" syntax
with st.form(key='my_form'):
	user_id = st.text_input("What's your name?", "John")
	submit_button = st.form_submit_button(label='Submit')
st.write("Welcome", user_id)


conversation_history = []

if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    if not OPENAI_API_KEY:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    client = OpenAI(api_key=OPENAI_API_KEY)
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        stream=True,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=100
    )
    answer = ''
    for chunk in response:
        chunk_content = chunk.choices[0].delta.content
        if chunk_content:
            answer += chunk_content
            print(chunk_content, end='', flush=True)
    st.session_state.messages.append({"role": "assistant", "content": answer})
    st.chat_message("assistant").write(answer)

if prompt.lower() in ['exit', 'quit', 'bye']:
    break